<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/pavelzw/GEFCom14-S-comparison/blob/main/gefcom14-s-deepar.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/pavelzw/GEFCom14-S-comparison/blob/main/gefcom14-s-deepar.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://minhaskamal.github.io/DownGit/#/home?url=https://github.com/pavelzw/GEFCom14-S-comparison/blob/main/gefcom14-s-deepar.ipynb&fileName=gefcom14-s-deepar"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Download data

In [1]:
!echo "Downloading GEFCom14-S..."
!rm -R data > /dev/null
!wget -O gefcom14.zip https://www.dropbox.com/s/pqenrr2mcvl0hk9/GEFCom2014.zip?dl=0
!unzip gefcom14 > /dev/null
!rm gefcom14.zip > /dev/null
!unzip GEFCom2014\ Data/GEFCom2014-S_V2.zip > /dev/null
!rm -R GEFCom2014\ Data > /dev/null
!mv Solar data > /dev/null
!echo "------------------------------"
!echo "Downloaded GEFCom14-S in data/"

# Install pip packages

In [2]:
!pip install mxnet
!pip install gluonts
!pip install hyperopt

In [3]:
import pandas as pd

task = 2
# only use surface solar radiation (169), surface thermal radiation (175) and top net solar radiation (178)
predictors = pd.read_csv(f'data/Task {task}/predictors{task}.csv', parse_dates=['TIMESTAMP'])\
    [['ZONEID', 'TIMESTAMP', 'VAR169', 'VAR175', 'VAR178']].set_index('TIMESTAMP')
train = pd.read_csv(f'data/Task {task}/train{task}.csv', parse_dates=['TIMESTAMP']).set_index('TIMESTAMP')
benchmark = pd.read_csv(f"data/Task {task-1}/benchmark{'0' + str(task-1) if task-1 < 10 else task}.csv",
                        parse_dates=['TIMESTAMP']).set_index('TIMESTAMP')

In [4]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

solar_plants = [train[train['ZONEID'] == i][['POWER']].rename({'POWER': f'ZONEID {i}'}, axis='columns')
                for i in [1,2,3]]
train_data = pd.concat(solar_plants, axis=1)

predictors_categories = [predictors[predictors['ZONEID'] == i][['VAR169', 'VAR175', 'VAR178']]
                             .rename({'VAR169': f'SURFACE SOLAR RADIATION {i}',
                                      'VAR175': f'SURFACE THERMAL RADIATION {i}',
                                      'VAR178': f'TOP NET SOLAR RADIATION {i}'}, axis='columns')
                         for i in [1,2,3]]
predictor_data = pd.concat(predictors_categories, axis=1)[:'2013-05-01 00:00']

# define the parameters of the dataset
gefcom14_metadata = {'num_series': 3,
                     'num_steps': 24 * (365 + 30),
                     'prediction_length': 24 * 30, # 1 month (April)
                     'freq': '1H',
                     'start': [pd.Timestamp("2012-04-01 01:00", freq='1H') for _ in range(3)]
                     }

# train_ds
targets = [train_data[:-gefcom14_metadata['prediction_length']][f'ZONEID {i}'].values for i in [1,2,3]]
starts = gefcom14_metadata['start']
features = [predictor_data[:-gefcom14_metadata['prediction_length']][[f'SURFACE SOLAR RADIATION {i}', f'SURFACE THERMAL RADIATION {i}', f'TOP NET SOLAR RADIATION {i}']].values.T for i in [1,2,3]]

train_ds = ListDataset([{
    FieldName.TARGET: target,
    FieldName.START: start,
    FieldName.FEAT_DYNAMIC_REAL: fdr,
    FieldName.FEAT_STATIC_CAT: [fsc]
  } for (target, start, fdr, fsc) in zip(targets, starts, features, [1,2,3])],
  freq=gefcom14_metadata['freq'])

# test_ds
targets = [train_data[f'ZONEID {i}'].values for i in [1,2,3]]
starts = gefcom14_metadata['start']
features = [predictor_data[[f'SURFACE SOLAR RADIATION {i}', f'SURFACE THERMAL RADIATION {i}', f'TOP NET SOLAR RADIATION {i}']].values.T for i in [1,2,3]]

test_ds = ListDataset([{
    FieldName.TARGET: target,
    FieldName.START: start,
    FieldName.FEAT_DYNAMIC_REAL: fdr,
    FieldName.FEAT_STATIC_CAT: [fsc]
  } for (target, start, fdr, fsc) in zip(targets, starts, features, [1,2,3])],
  freq=gefcom14_metadata['freq'])

In [5]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.mx.distribution import PiecewiseLinearOutput

def train(epochs=5,
          num_pieces=2,
          num_cells=40,
          num_layers=2,
          context_length=None,
          cell_type='lstm',
          dropoutcell_type='ZoneoutCell',
          # available: 'ZoneoutCell', 'RNNZoneoutCell', 'VariationalDropoutCell' or 'VariationalZoneoutCell'
          dropout_rate=0.1,
          use_feat_dynamic_real=False,
          alpha=0.0,
          beta=0.0):
    estimator = DeepAREstimator(freq=gefcom14_metadata['freq'],
                                prediction_length=gefcom14_metadata['prediction_length'],
                                num_cells=num_cells,
                                num_layers=num_layers,
                                dropout_rate=dropout_rate,
                                context_length=context_length,
                                cell_type=cell_type, # lstm, gru available
                                dropoutcell_type=dropoutcell_type,
                                use_feat_dynamic_real=use_feat_dynamic_real,
                                alpha=alpha,
                                beta=beta,
                                distr_output=PiecewiseLinearOutput(num_pieces=num_pieces), # SQF-RNN
                                trainer=Trainer(epochs=epochs))
    predictor = estimator.train(train_ds, validation_data=test_ds)
    return predictor

In [6]:
import numpy as np
from gluonts.evaluation.backtest import make_evaluation_predictions

def predict(predictor):
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,  # test dataset
        predictor=predictor,  # predictor
        num_samples=1000,  # number of sample paths we want for evaluation
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)

    zone_predictions = []
    for i, forecast in enumerate(forecasts):
        prediction = pd.concat([np.maximum(forecast.quantile_ts(p/100), 0)
                                for p in range(1, 100)], axis=1)\
            .rename(columns={p: str((p+1)/100) for p in range(99)})
        prediction.insert(0, 'ZONEID', i+1)
        prediction.index.name = 'TIMESTAMP'
        zone_predictions.append(prediction)

    predictions = pd.concat(zone_predictions)
    return tss, predictions

# Evaluate Loss

The loss function is the pinnball loss:
$$ L(q_a, y) = \begin{cases}
    (1-\frac{a}{100})(q_a - y), &\text{if } y < q_a \\
    \frac{a}{100}(y-q_a), &\text{if } y \geq q_a.
\end{cases} $$

The score is then averaged over all target quantiles for all time periods over the forecast horizon and for all zones.

In [7]:
def pinnball_loss(actual, prediction):
    actual = actual[...,None]

    percentiles = np.empty((actual.shape[0], 99))
    for i in range(1, 100):
        percentiles[:, i-1] = i
    loss = np.where(actual < prediction,
                    (1 - percentiles / 100) * (prediction - actual),
                    percentiles / 100 * (actual - prediction))
    return loss

def calculate_loss(tss, predictions):
    loss1 = np.mean(pinnball_loss(tss[0].values[-24*30:,0], predictions[predictions['ZONEID'] == 1].drop('ZONEID', axis=1)))
    loss2 = np.mean(pinnball_loss(tss[1].values[-24*30:,0], predictions[predictions['ZONEID'] == 2].drop('ZONEID', axis=1)))
    loss3 = np.mean(pinnball_loss(tss[2].values[-24*30:,0], predictions[predictions['ZONEID'] == 3].drop('ZONEID', axis=1)))
    loss = (loss1 + loss2 + loss3) / 3
    print(f"Loss of task {task}: {loss}")
    return loss

In [8]:
from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL

def objective(x):
    epochs = 5
    alpha = 0.0
    beta = 0.0
    num_pieces, num_cells, num_layers, context_length, \
    cell_type, dropoutcell_type, dropout_rate, use_feat_dynamic_real = x

    print(f'Testing with {x}')

    try:
        predictor = train(epochs, num_pieces, num_cells, num_layers, context_length, cell_type, dropoutcell_type,
                          dropout_rate, use_feat_dynamic_real, alpha, beta)
        tss, predictions = predict(predictor)
        loss = calculate_loss(tss, predictions)
        return {
            'loss': loss,
            'status': STATUS_OK
        }
    except UnboundLocalError:
        return {
            'status': STATUS_FAIL
        }

In [ ]:
from hyperopt import Trials

space = [
    hp.choice('num_pieces', range(2, 8)),
    hp.choice('num_cells', range(20, 100)),
    hp.choice('num_layers', range(2, 6)),
    hp.choice('context_length', range(24 * 60)),
    hp.choice('cell_type', ['lstm', 'gru']),
    hp.choice('dropoutcell_type', ['ZoneoutCell', 'RNNZoneoutCell', 'VariationalDropoutCell', 'VariationalZoneoutCell']),
    hp.uniform('dropout_rate', 0.01, 0.5),
    hp.choice('use_feat_dynamic_real', [False, True])
]

trials = Trials()
best = fmin(fn=objective,
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=48)
print(best)

In [22]:
trials.best_trial

{'state': 2,
 'tid': 30,
 'spec': None,
 'result': {'loss': 0.017649407492718944, 'status': 'ok'},
 'misc': {'tid': 30,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'cell_type': [30],
   'context_length': [30],
   'dropout_rate': [30],
   'dropoutcell_type': [30],
   'num_cells': [30],
   'num_layers': [30],
   'num_pieces': [30],
   'use_feat_dynamic_real': [30]},
  'vals': {'cell_type': [1],
   'context_length': [1419],
   'dropout_rate': [0.20388833937807294],
   'dropoutcell_type': [0],
   'num_cells': [62],
   'num_layers': [3],
   'num_pieces': [1],
   'use_feat_dynamic_real': [0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2021, 4, 23, 7, 59, 37, 62000),
 'refresh_time': datetime.datetime(2021, 4, 23, 9, 11, 40, 624000)}

In [23]:
trials.argmin

{'cell_type': 1,
 'context_length': 1419,
 'dropout_rate': 0.20388833937807294,
 'dropoutcell_type': 0,
 'num_cells': 62,
 'num_layers': 3,
 'num_pieces': 1,
 'use_feat_dynamic_real': 0}